In [18]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'llama-3-Korean-Bllossom-8B-Q4_K_M.gguf' to '.huggingface/download/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf.882dc22ffebe74c225373af08375017c4c529af439ccf6e0b5587188fb43d0bd.incomplete'

tokenizer_config.json:   0% 0.00/51.3k [00:00<?, ?B/s]

special_tokens_map.json: 100% 507/507 [00:00<00:00, 4.16MB/s]
Download complete. Moving file to special_tokens_map.json


.gitattributes: 100% 1.59k/1.59k [00:00<00:00, 16.5MB/s]
tokenizer_config.json: 100% 51.3k/51.3k [00:00<00:00, 5.37MB/s]
Download complete. Moving file to tokenizer_config.json
Download complete. Moving file to .gitattributes

README.md: 100% 7.61k/7.61k [00:00<00:00, 42.9MB/s]
Download complete. Moving file to README.md
Fetching 6 files:  17% 1/6 [00:00<00:02,  2.23it/s]
tokenizer.json:   0% 0.00/10.1M [00:00<?, ?B/s]

llama-3-Korean-Bllossom-8B-Q4_K_M.gguf:   0% 0.00/5.02G [00:00<?, ?B/s]
tokenizer.json: 100% 10.1M/10.1M [00:00<00:00, 20.4MB/s]
Download complete. Moving file

In [2]:
from llama_cpp import Llama
from transformers import AutoTokenizer

# 새로운 시도

### 질문 형식의 예시를 여러 개 보여주기

In [ ]:
model_id = 'Bllossom/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
PROMPT = '''Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성하는 것입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성하면 됩니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가) 싱가포르의 ‘태형’과 (나) 흥보전의 ‘매품’을 서로 비교하시오."
'''

In [ ]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 총 3개 생성해주세요. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"]
["답변 1"]

["질문 2"]
["답변 2"]

["질문 3"]
["답변 3"]
으로 부탁드립니다.

질문과 답변 세트를 3개 생성해주세요.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
[제시문 (가)]의 내용을 바탕으로, 어떤 이유로 사람들은 유행을 따르고 싶어하는 심리적 동기를 설명해주세요.

답변 1:
유행은 사람들이 자신들 주변의 다른 사람들과 일치하고 싶어하는 욕구와, 그들만의 개성을 표현하고자 하는 욕구가 복합적으로 얽혀 있는 것입니다. 따라서, 사람들은 상대들에게 인정과 안도감을 느끼기 위해 유행을 따르고 싶어합니다. 이는 그들이 주변 사람들에게 인정받고, 그들만의 스타일이 받아들여지고, 그들만의 개성이 존중받는 것을 의미합니다.

질문 2:
[제시문 (나)]의 내용을 바탕으로, 어떤 이유로 사람들은 옛 것들을 따라가고, 동시대 사람들을 모방하려는 심리적 동기를 설명해주세요.

답변 2:
옛것을 따르는 경향은 사람들의 변화를 평가절하하고, 현재의 규범을 중요시 하려는 경향이 함께 동반됩니다. 사람들은 과거를 존경하지 않고, 지금은 지금이 가장 중요한 것으로 여겨집니다. 또한, 사람들은 같은 시대를 사는 사람들과 연결하고자 하며, 동시대 사람들을 모방하려는 경향을 가지고 있습니다. 이는 사람들이 현재와 미래에 대한 불안감을 해소하고, 안정감을 느끼는 것을 의미합니다.

질문 3:
[제시문 (다)]의 내용을 바탕으로, 어떤 이유로 사람들은 언어 사용에서도 세대 차이와 연관이 있으며, 이를 어떻게 설명할 수 있을까요?

답변 3:
언어 사용에서도 세대 차이와 연관이 있습니다. 이는 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는데, 예를 들어, '엄마, 아빠'와 같은 말은 어린이들의 말이고, '자네, 댁'과 같은 단어는 어른들의 말이라는 것이 그러한 예입니다. 또한, 새로운 세대가 언어를 만들고 유행어를 만들기도 하는데, 이는 또 다른 세대 차이 중 하나인 젊은 세대의 특성을 대변합니다. 따라서, 언어 사용에서 세대 차이와 연관이 있는 것은 그들이 사회적인 존재로서의 역할을 통해 자연스럽게 발생하는 것입니다.


### 건님의 instruction과 결합

In [ ]:
qa_list = []

In [188]:
PROMPT = '''
Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 예시 제시문 세트로부터 예시 질문을 생성입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문들이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성합니다.
이 때 질문들은 제시문들 간의 연관성을 파악해 이를 물어보는 것입니다."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [189]:
test_text = '''다음 3개의 제시문을 보고, 질문과 그에 대한 답변세트를 3개 생성해야 합니다. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.

생성 형식은
["질문 1"
"질문 2"
"질문 3"
"답변 1"
"답변 2"
"답변 3"]
으로 해야 합니다.

질문은 반드시 명령조로 생성되어야 합니다.

이제 새로운 제시문 세트를 보여드리겠습니다.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령, ‘엄마, 아빠’와 같은 말은 어린이들의 말이고, ‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런 ‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [190]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":1,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Fetching 6 files: 100% 6/6 [00:00<00:00, 1114.27it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: MLP-KTLim/llama-3-Korean-Bllossom-70B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

### 영문입력

In [ ]:
qa_list = []

In [ ]:
PROMPT = '''
Role: "You are a teacher who runs a college entrance essay interview academy in South Korea."
Task: "Your main job is to generate example questions from a set of example statements.
I'm going to show you an example question. When you're given a new set of presentations, you figure out what they're about and then you create a similar set of questions.
At this time, the questions are to identify the relationship between the presentations and ask them."

Format :
"(다)의 사례들을 (가)와 (나)에 비추어 평가하시오.",
"(가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.",
"(가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.",
"(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오.",
"제시문 (나)를 바탕으로 기술에 대한 제시문 (가)와 제시문 (다)의 주장을 설명하시오.",
"제시문 (가)와 (나)의 내용을 바탕으로 제시문 (다)의 ᄀ을 평가하시오.",
"제시문 (가)와 (나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오.",
"(가)와 (나)에서 작가가 갖추어야 할 자질들을 찾아 차이점과 공통점을 설명하시오."
'''

In [ ]:
test_text = '''You need to look at the following 3 presentations and create 3 sets of questions and answers for them. Please delete all other unnecessary sentences.
All questions and answers should be in Korean, and concise, consisting of 1-2 sentences.
Each question should have a different form and content that should not overlap.
Please create three questions and three answers. As I said, when generating questions, the content should not imply or suggest the answers. When creating questions, it's important to consider the relationships between the contents of each prompt.
The question must be a commanding tone, and without honorifics.
Each question and answer should be related to all prompts.
Questions should be concise, and should not quote prompt.

The generation format is as follows:
["Question 1"]
"Question 2"
"Question 3"
"Answer 1"
"Answer 2"
[Answer 3]

Now, let me show you a new set of presentations.

[제시문 (가)]
유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

[제시문 (나)]
옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 (다)]
사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다.
                '''

In [ ]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-70B'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-70B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":500,
    "stop":["<|eot_id|>"],
    "top_p":0.85,
    "temperature":0.4,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
qa_list.append(generated)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Here are three sets of questions and answers for the three presents.

[질문 1]
유행은 누구든 따라 할 수 있는 것이 아니라는 점에 대해 설명하십시오.

답변 1>
유행은 바로 그 대중성을 지닌 요소 때문에 어느 누구든 따라 할 수 있는 것입니다. 유행은 수많은 사람들 사이에서 어떤 행동이나 취향이 보통으로 여겨지는 경우를 의미하는데, 이 행동이나 취향이 이미 많은 사람들이 하고 있기 때문에 누구든지 쉽게 따라 할 수 있습니다. 따라서 유행이 모든 사람들에게 개방적인 것이 아니라는 것은 틀린 생각입니다.

[질문 2]
유행이 나타나면 왜 사람들은 그 유행을 따라 하게 되는지 설명하십시오.

답변 2>
유행은 사람들이 자신이나 주변 사람들과 차별화되도록 해주고, 동시에 안도감과 만족감을 제공합니다. 따라서 사람들은 유행을 따르기 위해 노력하고, 그것이 그들을 주변 사람들과 동일시하지 않고 차별화된 개인으로 인정받도록 도와줍니다. 또한 유행은 사람들 사이의 상호작용과 커뮤니케이션을 원활하게 해주므로, 사람들은 이를 통해 유대감을 형성하고 즐거움을 느낄 수 있습니다.

[질문 3]
어떤 경우에서는 유행이 더 이상 존재하지 않을까요?

답변 3>
유행이 더 이상 존재하지 않는 경우는 두 가지입니다. 첫째, 일부 개인은 모방을 두려워하고, themselves로 구별되고 싶다면 유행을 따른다는 경우가 있습니다. 둘째, 모든 개인이 제각각의 특수한 존재가 되고자 하는 욕구가 결여된 경우입니다. 즉, 사람들이 다양성을 존중하지 않고 모두 같은 행동과 옷을 선택한다면, 그 때부터는 유행이 존재하지 않게 됩니다.


In [ ]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

[질문 1]
제시문 (가)의 내용을 기반으로, 어떤 사람들은 유행을 따른다는 이유가 무엇일까요? 

답변 1
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려고 하는 심리적인 이유와, 다른 사람들로부터 인정받고 동화되기 위한 만족감을 얻기 위해서이다. 

[질문 2]
제시문 (나)의 내용을 기반으로, 어떤 사람들은 왜 새로운 것을 숭배하고 동시대 사람들을 모방하느냐? 

답변 2
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 주요한 원리이다. 이 원칙들은 선조의 유산을 평가절하하고 현재의 규범을 중요시하려는 성향이 함께 동반된다. 

[질문 3]
제시문 (다)의 내용을 기반으로, 왜 사람들은 시대에 따라 다른 언어를 사용하느냐? 

답변 3
사람은 사회적인 존재이기 때문에 신분이나 역할에 따라 행동양식이 달라지며, 그 행동양식 중 하나가 언어 사용이다. 따라서, 세대 변화나 사회적 변화에 따라 사용되는 언어도 변화하게 된다. 특히, 젊은 세대는 새로운 유행어와 신조어를 만들어내며 유행에 민감하고 변화를 쉽게 받아들이는 경향이 때문이다.
**************************************************
[질문 1]
제시문 (가)의 내용을 바탕으로, 유행이 왜 모든 개인을 유혹하는 것인지 묻습니다.

[답변 1]
유행은 바로 다른 사람들과 나를 구분할 수 있는 좋은 방법이 되기 때문입니다. 모든 개인이 제각각 특수한 존재가 되고 싶어하는 욕구와 동시에 모방을 회피하고자 하는 심리적인 요인이 결합되어 있기 때문입니다. 

[질문 2]
제시문 (나)의 내용을 바탕으로, 왜 사람들은 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시기를 특징으로 만들게 되었을까요?

[답변 2]
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시기를 이끌어내는 두 가지 중요한 원리입니다. 이 원소들은 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 육안으로 결합됩니다. 

[질문 3]
제

# 0624 모델 구현

필요한 것

- 질문 평가 metric

- 생성할 질문의 개수 다르게 input 받기

- 제시문은 통합으로 넣기

- 프롬프트와 질문 생성을 나눠서 볼 수 있도록

In [96]:
import pandas as pd
import random
qa_list = []
df_list = []
columns = ['질문 개수', '프롬프트', 'instruction', '생성 질문']

In [242]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

### 연세대 2023 인문계 학종전형
new_presentation= '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''

In [243]:
if num_of_question == 3 :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"]'''
elif num_of_question == 4 :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"]'''

else :
  format = '''
  ["질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"]'''

In [244]:
PROMPT = '''
Role : "당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다."
Task : "당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
예시 질문을 보여드리겠습니다. 당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 유사한 질문을 생성합니다.
모든 질문은 반드시 "~하시오."로 끝나야 합니다."

Format :
"환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오.",
"글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오.",
"제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오.",
"글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."
'''

In [245]:
test_text = f'''다음 제시문을 보고, 질문과 답변 세트를 {num_of_question}개 생성해야 합니다. 다른 불필요한 문장은 모두 삭제해 주세요.
질문과 답변은 모두 한국어로, 1~2문장으로 구성해주세요.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 합니다. 질문 부분에는 질문 자체만 생성해주세요.
모든 질문은 반드시 "~하시오."로 끝나야 합니다.

생성 형식은
{format}
으로 해야 합니다.

이제 새로운 제시문을 보여드리겠습니다.
{new_presentation}
                '''

In [246]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":1000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.8,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list.append(generated)
df_list.append([num_of_question, PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 3319.59it/s]
/content


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1: 유행은 누구를 따라가는 것이 아니라, 무엇을 따라가는 것일까요? 
답변 1: 유행은 어떤 행동이나 패션, 음악 등을 많은 사람들이 특정 시기에 따라 하는 것입니다. 따라서, 누구를 따라가는 것이 아니라, 어떤 것을 따라가는 것입니다.

질문 2: 왜 사람들은 유행에 민감하고 새로운 변화에 쉽게 적응하는지?
답변 2: 사람들은 자아표현과 개성을 중요시하기 때문에 새로운 변화나 유행에 민감합니다. 또한, 집합체 안에서 인정받고 동화되기 위한 욕구가 있기 때문에 새로운 것에 대한 호기심과 열정이 있습니다.

질문 3: 어떤 요인들이 유행의 변화를 이끌어내는가요?
답변 3: 유행의 변화는 크게 세 가지 요인에 의해 일어납니다. 첫째, 대중들의 소비욕구와 제작자의 창작욕구는 끝없이 이어지는 것입니다. 둘째, 마케팅 전문가들과 디자이너들은 유행의 변화를 예측하고, 이에 맞춰 제품 및 광고를 출시합니다. 셋째, 언론 매체와 SNS, 블로그 등에서 유행에 대한 기사가 많이 전해지면서 유행이 형성됩니다.
질문 개수 : 3


In [247]:
for a in range(len(qa_list)):
  print(qa_list[a])
  print('*'*50)

질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?
답변 1: 유행은 바로 주변 사람들과 비슷하게 행동하고 싶은 욕구와 자신을 표현하고 싶은 성격 욕구가 복합적으로 얽혀 있는 것입니다. 따라서, 유행을 따른다는 것은 다른 사람들과 비슷하게 행동하고, 그들보다 더 잘 보여지기 위한 노력이 필요하다는 것을 의미합니다.

질문 2: 유행이 나타나면 왜 모든 사람들이 그 유행을 따라하게 되는 것일까요?
답변 2: 유행은 현재 상황에서 유행하는 것이 자랑거리와 일종의 안도감이 되기도 하기 때문입니다. 또한, 다른 사람들과 구별되는 만족감을 얻으려고 하는 심리적 요인도 있습니다. 이러한 이유로, 사람들은 유행을 따라하기 위해 노력하고 있습니다.

질문 3: 유행이 나타나면 어떤 영향을 끼치는가?
답변 3: 유행은 사회 생활 전반에 큰 영향을 미칩니다. 예를 들어, 유행하는 옷이나 음악 등을 통해 사람들의 패션 감각과 음악 취향 등이 변화하게 됩니다. 또한, 유행을 따라하면서 사람들 간의 공동체적인 결합력이 형성되기도 합니다.

질문 4: 마지막으로, 왜 유행이 언제나 변화하는 것일까요?
답변 4: 유행은 사람들이 새로운 것을 탐닉하고, 변화를 쉽게 받아들일 수 있는 심리적 특성 때문입니다. 또한, 어떤 유행이 있더라도 사람들의 다양성과 개성 때문에 그 유행을 모두 따라가는 것은 불가능합니다. 따라서, 유행은 끊임없이 변화하며, 새로운 유행이 등장하여 이전 유행을 밀어내는 것이 일반적입니다.
**************************************************
질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?
답변 1: 유행은 바로 주변 사람들과 비슷하게 행동하고 싶은 욕구와 자신을 표현하고 싶은 성격 욕구가 복합적으로 얽혀 있는 것입니다. 따라서, 유행을 따른다는 것은 다른 사람들과 비슷하게 행동하고, 그들처럼 옷을 입고, 행동을 하는 것을 의미합니다.

질문 2: 유행이 나타나면 왜 모든 사람들이 그 유

In [248]:
df = pd.DataFrame(df_list, columns=columns)
df

,질문 개수,프롬프트,instruction,생성 질문
0,4,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 4개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?\n답변 1..."
1,5,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 5개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?\n답변 1..."
2,5,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 5개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이 아니라는 것을..."
3,4,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 4개 생성해야 합니다. 다른 불필요한 문장...",질문 1: 유행은 누구의 욕구나 취향에 의해 형성되는지 설명하시오?\n답변 1: 유...
4,3,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 3개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이 아니라는 주장..."
5,5,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 5개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유념해야 할 점에 대한, 글에서 언급된 두 가지 다른 입장을 비교하십시오..."
6,3,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 3개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?\n답변 1..."
7,3,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 3개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?\n답변 1..."
8,3,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 3개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오.\n답변 ..."
9,5,"\nRole : ""당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입...","다음 제시문을 보고, 질문과 답변 세트를 5개 생성해야 합니다. 다른 불필요한 문장...","질문 1: 유행은 누구를 따라가는 것이 아니라, 누구를 모방하는 것이냐?\n답변 1..."


In [249]:
df.to_csv('0624 시도.csv', encoding = 'cp949')